# Chapter 26: Key Design Patterns in C#

Design patterns are reusable solutions to common problems in software design. They are not finished pieces of code but templates that you can adapt to your specific situation. Patterns represent best practices evolved over time by experienced software developers.

In this chapter, you'll learn:

- What design patterns are and why they're useful.
- The three categories of patterns: **Creational**, **Structural**, and **Behavioral**.
- Key patterns in each category, with practical C# examples:
  - **Creational**: Singleton, Factory Method, Builder.
  - **Structural**: Adapter, Decorator, Facade.
  - **Behavioral**: Strategy, Observer, Command.
- When to apply each pattern and when to avoid them.
- How these patterns relate to modern C# features (e.g., dependency injection, events).

By the end, you'll be able to recognize situations where a pattern can improve your code's flexibility, maintainability, and clarity.

---

## 26.1 What Are Design Patterns?

Design patterns are typical solutions to common problems in software design. Each pattern is like a blueprint that you can customize to solve a particular design problem in your code.

Patterns are not invented; they are discovered by observing what works in practice. They capture the experience of developers and provide a common vocabulary for discussing design.

The classic book *Design Patterns: Elements of Reusable Object‑Oriented Software* by the "Gang of Four" (GoF) introduced 23 patterns. Many of them remain relevant today.

### Benefits of Using Patterns

- **Reusability** – solve recurring problems with proven solutions.
- **Communication** – developers can use pattern names to convey complex ideas quickly (e.g., "we need a Singleton here").
- **Flexibility** – patterns often lead to designs that are easier to change and extend.
- **Avoid reinventing the wheel** – leverage collective experience.

However, patterns should not be applied blindly. They are tools, not rules. Over‑engineering with patterns can make code more complex than necessary.

---

## 26.2 Creational Patterns

Creational patterns deal with object creation mechanisms, trying to create objects in a manner suitable to the situation. The basic form of object creation could result in design problems or added complexity. Creational patterns solve this by controlling the creation process.

### 26.2.1 Singleton

**Intent:** Ensure a class has only one instance and provide a global point of access to it.

**When to use:** You need exactly one instance of a class, such as a logging service, configuration manager, or thread pool.

**Implementation in C#:**

The classic singleton uses a private constructor, a static field, and a static method.

```csharp
public sealed class ConfigurationManager
{
    private static readonly ConfigurationManager _instance = new ConfigurationManager();
    private ConfigurationManager() { } // private constructor

    public static ConfigurationManager Instance => _instance;

    public string GetSetting(string key) { /* ... */ }
}
```

This is thread‑safe and lazy‑initialized because the static constructor runs once when the type is first used.

**Variation with Lazy<T>:**

```csharp
public sealed class Logger
{
    private static readonly Lazy<Logger> _lazy = new Lazy<Logger>(() => new Logger());
    private Logger() { }
    public static Logger Instance => _lazy.Value;
}
```

**Pitfalls:**

- Singletons can make unit testing difficult because they introduce global state.
- Consider dependency injection instead; often a single instance can be registered as a singleton in the DI container, which gives you the same benefit without the hard coupling.

### 26.2.2 Factory Method

**Intent:** Define an interface for creating an object, but let subclasses decide which class to instantiate. Factory Method lets a class defer instantiation to subclasses.

**When to use:** A class cannot anticipate the class of objects it must create. You want to decouple the client code from the concrete product classes.

**Example:**

Suppose you have an abstract `Document` class and concrete `PdfDocument`, `WordDocument`. A `DocumentCreator` base class defines a factory method `CreateDocument()`.

```csharp
public abstract class Document
{
    public abstract void Open();
}

public class PdfDocument : Document
{
    public override void Open() => Console.WriteLine("Opening PDF...");
}

public class WordDocument : Document
{
    public override void Open() => Console.WriteLine("Opening Word document...");
}

public abstract class DocumentCreator
{
    public abstract Document CreateDocument();

    public void EditDocument()
    {
        var doc = CreateDocument();
        doc.Open();
        // additional operations
    }
}

public class PdfCreator : DocumentCreator
{
    public override Document CreateDocument() => new PdfDocument();
}

public class WordCreator : DocumentCreator
{
    public override Document CreateDocument() => new WordDocument();
}

// Usage
DocumentCreator creator = new PdfCreator();
creator.EditDocument();
```

Factory Method is a cornerstone of many frameworks. In modern C#, you might also use a `Func<T>` delegate or a DI container to achieve similar decoupling.

### 26.2.3 Builder

**Intent:** Separate the construction of a complex object from its representation so that the same construction process can create different representations.

**When to use:** An object requires many configuration steps or optional parameters. The telescoping constructor pattern (many overloads) becomes unwieldy.

**Example:**

Building a `Computer` with many optional parts.

```csharp
public class Computer
{
    public string CPU { get; set; }
    public string RAM { get; set; }
    public string Storage { get; set; }
    public string GraphicsCard { get; set; }

    public override string ToString() =>
        $"CPU: {CPU}, RAM: {RAM}, Storage: {Storage}, Graphics: {GraphicsCard}";
}

public interface IComputerBuilder
{
    void SetCPU(string cpu);
    void SetRAM(string ram);
    void SetStorage(string storage);
    void SetGraphicsCard(string graphics);
    Computer GetComputer();
}

public class GamingComputerBuilder : IComputerBuilder
{
    private Computer _computer = new Computer();

    public void SetCPU(string cpu) => _computer.CPU = cpu;
    public void SetRAM(string ram) => _computer.RAM = ram;
    public void SetStorage(string storage) => _computer.Storage = storage;
    public void SetGraphicsCard(string graphics) => _computer.GraphicsCard = graphics;
    public Computer GetComputer() => _computer;
}

public class Director
{
    private IComputerBuilder _builder;

    public Director(IComputerBuilder builder) => _builder = builder;

    public void ConstructGamingPC()
    {
        _builder.SetCPU("Intel i9");
        _builder.SetRAM("32GB DDR5");
        _builder.SetStorage("1TB NVMe SSD");
        _builder.SetGraphicsCard("RTX 4090");
    }
}

// Usage
var builder = new GamingComputerBuilder();
var director = new Director(builder);
director.ConstructGamingPC();
Computer computer = builder.GetComputer();
Console.WriteLine(computer);
```

In modern C#, you might use a fluent interface with method chaining, which is a variation of the builder pattern (often called a "Fluent Builder").

```csharp
public class ComputerBuilder
{
    private Computer _computer = new Computer();
    public ComputerBuilder WithCPU(string cpu) { _computer.CPU = cpu; return this; }
    public ComputerBuilder WithRAM(string ram) { _computer.RAM = ram; return this; }
    public Computer Build() => _computer;
}

// Usage
var computer = new ComputerBuilder()
    .WithCPU("Intel i7")
    .WithRAM("16GB")
    .Build();
```

---

## 26.3 Structural Patterns

Structural patterns explain how to assemble objects and classes into larger structures while keeping the structures flexible and efficient.

### 26.3.1 Adapter

**Intent:** Convert the interface of a class into another interface clients expect. Adapter lets classes work together that couldn't otherwise because of incompatible interfaces.

**When to use:** You have an existing class that provides the functionality you need, but its interface is not compatible with your current system.

**Example:**

Suppose you have a third‑party library that provides weather data via a `GetWeather(string city)` method, but your system expects an `IWeatherService` with a method `GetTemperature(string city)`.

```csharp
// Target interface
public interface IWeatherService
{
    double GetTemperature(string city);
}

// Adaptee (third‑party)
public class ThirdPartyWeatherAPI
{
    public double GetWeather(string city) => 25.0; // returns temperature
}

// Adapter
public class WeatherAdapter : IWeatherService
{
    private readonly ThirdPartyWeatherAPI _api;
    public WeatherAdapter(ThirdPartyWeatherAPI api) => _api = api;

    public double GetTemperature(string city) => _api.GetWeather(city);
}

// Usage
ThirdPartyWeatherAPI api = new ThirdPartyWeatherAPI();
IWeatherService service = new WeatherAdapter(api);
double temp = service.GetTemperature("London");
```

Adapters are often used in integration scenarios where you need to plug external components into your application.

### 26.3.2 Decorator

**Intent:** Attach additional responsibilities to an object dynamically. Decorators provide a flexible alternative to subclassing for extending functionality.

**When to use:** You need to add behavior to individual objects, not to entire classes. For example, adding logging, caching, or compression to a data stream.

**Example:**

A simple `INotifier` interface with a concrete `EmailNotifier`. We want to add optional SMS and Facebook notifications without modifying the existing class.

```csharp
public interface INotifier
{
    void Send(string message);
}

public class EmailNotifier : INotifier
{
    public void Send(string message) => Console.WriteLine($"Email: {message}");
}

// Base decorator
public abstract class NotifierDecorator : INotifier
{
    protected INotifier _notifier;
    public NotifierDecorator(INotifier notifier) => _notifier = notifier;
    public virtual void Send(string message) => _notifier.Send(message);
}

// Concrete decorators
public class SmsNotifierDecorator : NotifierDecorator
{
    public SmsNotifierDecorator(INotifier notifier) : base(notifier) { }
    public override void Send(string message)
    {
        base.Send(message);
        Console.WriteLine($"SMS: {message}");
    }
}

public class FacebookNotifierDecorator : NotifierDecorator
{
    public FacebookNotifierDecorator(INotifier notifier) : base(notifier) { }
    public override void Send(string message)
    {
        base.Send(message);
        Console.WriteLine($"Facebook: {message}");
    }
}

// Usage
INotifier notifier = new EmailNotifier();
notifier = new SmsNotifierDecorator(notifier);
notifier = new FacebookNotifierDecorator(notifier);
notifier.Send("Hello!"); // sends via email, SMS, and Facebook
```

Decorators are powerful because you can combine them in any order at runtime.

### 26.3.3 Facade

**Intent:** Provide a unified interface to a set of interfaces in a subsystem. Facade defines a higher‑level interface that makes the subsystem easier to use.

**When to use:** You have a complex subsystem with many interdependent classes, and you want to provide a simple entry point for common tasks.

**Example:**

Consider a home theater system with many components: `Amplifier`, `DvdPlayer`, `Projector`, `Lights`. Instead of having the client interact with all of them, you create a `HomeTheaterFacade`.

```csharp
public class Amplifier { public void On() { } public void SetVolume(int level) { } }
public class DvdPlayer { public void On() { } public void Play(string movie) { } }
public class Projector { public void On() { } public void SetInput(DvdPlayer player) { } }
public class Lights { public void Dim(int level) { } }

public class HomeTheaterFacade
{
    private Amplifier _amp;
    private DvdPlayer _dvd;
    private Projector _projector;
    private Lights _lights;

    public HomeTheaterFacade(Amplifier amp, DvdPlayer dvd, Projector projector, Lights lights)
    {
        _amp = amp;
        _dvd = dvd;
        _projector = projector;
        _lights = lights;
    }

    public void WatchMovie(string movie)
    {
        _lights.Dim(10);
        _amp.On();
        _amp.SetVolume(20);
        _projector.On();
        _projector.SetInput(_dvd);
        _dvd.On();
        _dvd.Play(movie);
    }
}

// Usage
var facade = new HomeTheaterFacade(new Amplifier(), new DvdPlayer(), new Projector(), new Lights());
facade.WatchMovie("Inception");
```

The facade simplifies the client's interaction with the subsystem.

---

## 26.4 Behavioral Patterns

Behavioral patterns are concerned with algorithms and the assignment of responsibilities between objects.

### 26.4.1 Strategy

**Intent:** Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.

**When to use:** You have multiple ways to perform an operation, and you want to select the algorithm at runtime.

**Example:**

A shipping calculator that uses different strategies for different carriers.

```csharp
public interface IShippingStrategy
{
    decimal Calculate(Order order);
}

public class FedExShippingStrategy : IShippingStrategy
{
    public decimal Calculate(Order order) => 5.00m + (order.Total * 0.1m);
}

public class UPSShippingStrategy : IShippingStrategy
{
    public decimal Calculate(Order order) => 8.00m + (order.Total * 0.05m);
}

public class USPSShippingStrategy : IShippingStrategy
{
    public decimal Calculate(Order order) => 3.00m;
}

public class Order
{
    public decimal Total { get; set; }
    public IShippingStrategy ShippingStrategy { get; set; }

    public decimal GetShippingCost() => ShippingStrategy?.Calculate(this) ?? 0;
}

// Usage
Order order = new Order { Total = 100 };
order.ShippingStrategy = new FedExShippingStrategy();
Console.WriteLine(order.GetShippingCost()); // 15
```

Strategy pattern is often used with dependency injection to inject the desired strategy.

### 26.4.2 Observer

**Intent:** Define a one‑to‑many dependency between objects so that when one object changes state, all its dependents are notified and updated automatically.

**When to use:** You have a subject that needs to notify multiple observers about state changes. The classic example is UI updates when data changes.

In C#, events are a built‑in implementation of the Observer pattern.

```csharp
public class Stock
{
    private string _symbol;
    private decimal _price;
    public event EventHandler<PriceChangedEventArgs> PriceChanged;

    public Stock(string symbol, decimal price)
    {
        _symbol = symbol;
        _price = price;
    }

    public decimal Price
    {
        get => _price;
        set
        {
            if (_price != value)
            {
                _price = value;
                OnPriceChanged(new PriceChangedEventArgs(_symbol, value));
            }
        }
    }

    protected virtual void OnPriceChanged(PriceChangedEventArgs e)
    {
        PriceChanged?.Invoke(this, e);
    }
}

public class PriceChangedEventArgs : EventArgs
{
    public string Symbol { get; }
    public decimal NewPrice { get; }
    public PriceChangedEventArgs(string symbol, decimal newPrice)
    {
        Symbol = symbol;
        NewPrice = newPrice;
    }
}

public class Investor
{
    public string Name { get; set; }
    public void Subscribe(Stock stock)
    {
        stock.PriceChanged += OnPriceChanged;
    }

    private void OnPriceChanged(object sender, PriceChangedEventArgs e)
    {
        Console.WriteLine($"Notified {Name} that {e.Symbol} price changed to {e.NewPrice:C}");
    }
}

// Usage
Stock msft = new Stock("MSFT", 100m);
Investor alice = new Investor { Name = "Alice" };
alice.Subscribe(msft);

msft.Price = 105m; // triggers event
```

The observer pattern promotes loose coupling between the subject and observers.

### 26.4.3 Command

**Intent:** Encapsulate a request as an object, thereby letting you parameterize clients with different requests, queue or log requests, and support undoable operations.

**When to use:** You need to issue requests to objects without knowing anything about the operation being requested or the receiver of the request.

**Example:**

A simple text editor with commands for copy, paste, and undo.

```csharp
public interface ICommand
{
    void Execute();
    void Undo();
}

public class TextEditor
{
    public string Text { get; set; }
    public int SelectionStart { get; set; }
    public int SelectionLength { get; set; }
}

public class CopyCommand : ICommand
{
    private readonly TextEditor _editor;
    private string _copiedText;

    public CopyCommand(TextEditor editor) => _editor = editor;

    public void Execute()
    {
        _copiedText = _editor.Text.Substring(_editor.SelectionStart, _editor.SelectionLength);
        Console.WriteLine($"Copied: {_copiedText}");
    }

    public void Undo() => Console.WriteLine("Undo copy (nothing to undo)");
}

public class PasteCommand : ICommand
{
    private readonly TextEditor _editor;
    private string _previousText;

    public PasteCommand(TextEditor editor) => _editor = editor;

    public void Execute()
    {
        _previousText = _editor.Text;
        _editor.Text = _editor.Text.Insert(_editor.SelectionStart, "PASTED");
        Console.WriteLine($"Pasted. New text: {_editor.Text}");
    }

    public void Undo()
    {
        _editor.Text = _previousText;
        Console.WriteLine("Undo paste.");
    }
}

// Invoker
public class CommandInvoker
{
    private readonly Stack<ICommand> _undoStack = new Stack<ICommand>();

    public void ExecuteCommand(ICommand command)
    {
        command.Execute();
        _undoStack.Push(command);
    }

    public void Undo()
    {
        if (_undoStack.Count > 0)
        {
            var command = _undoStack.Pop();
            command.Undo();
        }
    }
}

// Usage
TextEditor editor = new TextEditor { Text = "Hello world", SelectionStart = 6, SelectionLength = 5 };
CommandInvoker invoker = new CommandInvoker();

invoker.ExecuteCommand(new CopyCommand(editor));
invoker.ExecuteCommand(new PasteCommand(editor));
invoker.Undo(); // undoes paste
```

The command pattern is used in undo/redo systems, task queues, and menu actions.

---

## 26.5 When to Use (and Not Use) Design Patterns

Patterns are powerful, but they come with complexity. Apply them when they solve a real problem, not just because you can.

### Signs You Might Need a Pattern

- You find yourself writing similar code in many places.
- Your code becomes hard to change because of tight coupling.
- You need to provide flexibility for future extensions.
- You have a common problem that patterns are known to solve.

### Signs of Over‑Engineering

- The pattern adds more complexity than the problem warrants.
- You're using a pattern because you read about it, not because you need it.
- The code becomes harder to understand for other developers.

**Rule of thumb:** Prefer simple, clear solutions first. Introduce patterns when the code demands it. Patterns are most valuable when they make the code easier to maintain and extend.

---

## 26.6 Patterns and Modern C#

Many patterns are built into the language or framework:

- **Singleton** – often replaced by DI container singleton lifetimes.
- **Factory** – can be a simple `Func<T>` delegate.
- **Observer** – `event` keyword and `INotifyPropertyChanged`.
- **Strategy** – often implemented with delegates/lambdas.
- **Command** – `Action` and `Func` can serve as lightweight commands.

However, understanding the pattern helps you design better solutions even when using these language features.

---

## 26.7 Chapter Summary

In this chapter, you've explored essential design patterns that appear frequently in C# applications:

- **Creational patterns** (Singleton, Factory Method, Builder) manage object creation.
- **Structural patterns** (Adapter, Decorator, Facade) compose objects and classes.
- **Behavioral patterns** (Strategy, Observer, Command) define communication between objects.
- You saw practical C# examples for each pattern.
- You learned when to apply patterns and when to avoid them.

Design patterns give you a shared vocabulary and proven solutions. As you gain experience, you'll recognize opportunities to use them naturally, making your code more flexible and maintainable.

In the next chapter, **Domain‑Driven Design (DDD) Concepts**, you'll learn how to model complex business domains using DDD principles. We'll cover entities, value objects, aggregates, repositories, and domain events – and see how they relate to patterns you already know.

**Exercises:**

1. Implement a simple logging system using the Singleton pattern. Then refactor it to use dependency injection with a DI container. Compare the testability.
2. Use the Factory Method pattern to create different types of `Character` in a game (Warrior, Mage, Archer). Add a `GameLevel` class that uses the factory.
3. Create a decorator chain for a `Coffee` interface: start with a base coffee, then add decorators for milk, sugar, whipped cream. Calculate total cost.
4. Build a simple shopping cart that uses the Strategy pattern for discount calculation (e.g., no discount, percentage discount, fixed amount discount).
5. Implement an undoable text editor using the Command pattern. Support at least typing and deleting.

Now, get ready to dive into Domain‑Driven Design in Chapter 27!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='25. designing_for_performance.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='27. domain_driven_design_concepts.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
